In [ ]:
# hello!!
# 


In [2]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np

In [3]:
import nltk
# nltk.download('stopwords') #run once 
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

from emot.emo_unicode import UNICODE_EMOJI

In [4]:
from textblob import TextBlob

In [5]:
import re

In [11]:
count_hash = {}
count_at = {}

def count_tags(tweet_text):
    hash_tag = re.findall(r"#(\w+)", tweet_text)
    at_tag = re.findall(r"@(\w+)", tweet_text)
    
    if(len(hash_tag)!=0):
        for i in hash_tag: 
            if i not in count_hash.keys():
                count_hash[i] = 1
            else:
                count_hash[i] += 1    
        
        for i in at_tag: 
            if i not in count_at.keys():
                count_at[i] = 1
            else:
                count_at[i] += 1   
        

In [22]:
#search ??
query = ["Donald Trump lang:en", "Joe Biden lang:en"]

tweets = []

for n in range(2):
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query[n]).get_items()):   
        if i>1000:
            break

        else:
            # extract specific tweets
            #particular length?
            #retweet ko lena?
            tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
            count_tags(tweet.content)

df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
df.to_csv('sentiment.csv')

/tmp/ipykernel_27887/4194981117.py:15: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
/tmp/ipykernel_27887/4194981117.py:16: FutureWarning: content is deprecated, use rawContent instead
  count_tags(tweet.content)


In [6]:
# df = pd.read_csv('sentiment.csv')

In [13]:

# print(count_at)
# print()
# print(count_hash)

top_hash = []
top_at = []

value_key_pairs = ((value, key) for (key,value) in count_hash.items())
sorted_value_key_pairs1 = sorted(value_key_pairs, reverse=True)

dftemp = pd.DataFrame(sorted_value_key_pairs1, columns=["number", "top_hash_tags"])
dftemp.to_csv("hash_tags.csv", index=False)

value_key_pairs = ((value, key) for (key,value) in count_at.items())
sorted_value_key_pairs2 = sorted(value_key_pairs, reverse=True)

dftemp = pd.DataFrame(sorted_value_key_pairs2, columns=["number", "top_at_tags"])
dftemp.to_csv("at_tags.csv", index=False)

#problem if deleting
# def max_five(count_hash, count_at):
#     for i in range (20):
#         Keymax = max(zip(count_at.values(), count_at.keys()))[1]
#         # print(zip(count_at.values(), count_at.keys()))
#         top_at.append((Keymax, count_at[Keymax]))
#         del count_at[Keymax]
    
#     for i in range (20):
#         Keymax = max(zip(count_hash.values(), count_hash.keys()))[1]
#         # print(zip(count_hash.values(), count_hash.keys()))
#         top_hash.append((Keymax, count_hash[Keymax]))
#         del count_hash[Keymax]

# max_five(count_hash, count_at)
# max_five(count_at, top_at)

# print()
# print(top_hash)
# print()
# print(top_at)

In [14]:
eng_stop_words = list(stopwords.words('english'))

# stopwordsHindi=[]
# with open("hindiStopWords.txt") as f:
#     totalData=f.readlines()
#     for i in range(len(totalData)):
#         stopwordsHindi.append(totalData[i])

emoji = list(UNICODE_EMOJI.keys())

In [15]:
# regex


def filtering(text):
    text = text.lower()

    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())
    
    # print(text)
    
    # remove stop words
    tokens = word_tokenize(text)  
    filtered_words = [w for w in tokens if w not in eng_stop_words]
    # filtered_words = [w for w in filtered_words if w not in stopwordsHindi]
    filtered_words = [w for w in filtered_words if w not in emoji]

    lemmatizer = WordNetLemmatizer() 
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = " ".join(lemma_words)

    # print()
    # print(text)
    
    return text
    
    



# filtering("@kvenkat1680 @SecularFringe @GemsOfBollywood Those will loss Who were faking despite of political alignments. As per Hindenberg, Adani manipulated &; used a fake assets scene and https://www.sdhvb.com he is getting rewards for his deeds, no surprise. As per your views the super hit movies are unable to stay on screen for more than two weeks. #Truth")


In [16]:
df = pd.read_csv('sentiment.csv')
df['Processed_Tweets'] = df['tweet'].apply(filtering)

In [19]:
def polarity(tweet):
    return TextBlob(tweet).sentiment.polarity


df['Polarity'] = df['Processed_Tweets'].apply(polarity)
df.head(20)



0.35


,Unnamed: 0,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet,Processed_Tweets,Polarity
0,0,2023-02-21 12:13:49+00:00,1628005068862521348,https://twitter.com/justtjadaxo/status/1628005...,justtjadaxo,Twitter for iPhone,NaN,Donald Trump acted like this every single day....,0,0,donald trump acted like every single day pleas...,-0.071429
1,1,2023-02-21 12:13:40+00:00,1628005030442995713,https://twitter.com/koh_e_nur_Laila/status/162...,koh_e_nur_Laila,Twitter for Android,Hidden Village of Mist,Mr. Donald Trump for the President 2024.\n\n#B...,0,0,mr donald trump president 2024,0.000000
2,2,2023-02-21 12:13:23+00:00,1628004957428252672,https://twitter.com/NickDoesLife1/status/16280...,NickDoesLife1,Twitter Web App,NaN,@JHenzi @AlliRaine22 The last time there was a...,0,0,last time backlash leftwing extremism got 4 ye...,0.195238
3,3,2023-02-21 12:13:12+00:00,1628004910439747584,https://twitter.com/WeVote2020/status/16280049...,WeVote2020,"Cheap Bots, Done Quick!",USA,Conservatives are rallying behind Donald Trump 👎,0,0,conservative rallying behind donald trump,-0.400000
4,4,2023-02-21 12:12:56+00:00,1628004845381644290,https://twitter.com/randomtrump1/status/162800...,randomtrump1,Random Trump,NaN,Random Trump https://t.co/Na07Zc4bQP - #random...,0,0,random trump,-0.500000
5,5,2023-02-21 12:12:51+00:00,1628004822770151430,https://twitter.com/JeanMarieAntiDz/status/162...,JeanMarieAntiDz,Twitter for Android,Algérie française,@mehdi_arafa @lambertiste Historiquement la dr...,0,0,arafa historiquement la droite toujours eu de ...,0.000000
6,6,2023-02-21 12:12:36+00:00,1628004760816177154,https://twitter.com/PaulDra0333/status/1628004...,PaulDra0333,Twitter for Android,NaN,@RonFilipkowski @mikest1r000 It is Democrats f...,0,0,democrat fault even fault everything disaster ...,0.000000
7,7,2023-02-21 12:11:34+00:00,1628004500450557954,https://twitter.com/j_itinerante/status/162800...,j_itinerante,Twitter for Android,NaN,@dimoralesj Los medios de comunicación en su c...,0,0,los medios de comunicaci n en su contra sea cu...,0.000000
8,8,2023-02-21 12:11:30+00:00,1628004483438374913,https://twitter.com/HershAlan/status/162800448...,HershAlan,Twitter Web App,NaN,Joe Biden vs. Donald tRump: https://t.co/fSDzT...,0,0,joe biden v donald trump,0.000000
9,9,2023-02-21 12:11:24+00:00,1628004457056198658,https://twitter.com/LIpsychicmedium/status/162...,LIpsychicmedium,Twitter for iPad,New York,The road Donald Trump can’t go down https://t....,0,0,road donald trump go via,0.000000


In [18]:
df.to_csv('sentiment.csv', mode = 'w')

In [75]:
#  polarity

In [76]:
# election oriented

# like search 2024 elections trump biden .....

# removing emotionless words

# freq of top words
# segregate according to state and freq of top words

# overall polarity
# segregate according to state and polarity

# election advertising